In [1]:
import glob
import torch
import pandas as pd


from core.logger_reader import LoggerReader
from collections import defaultdict, OrderedDict
from pathlib import Path

In [5]:
root = Path('results')

In [6]:
args_white_list = {
#  'output_root_dir',
#  'cv_run_num',
#  'ds_train_name',
#  'ds_test_name',
#  'ds_normalization',
#  'num_train_samples',
#  'num_augmentations',
#  'typ_augmentation',
 'num_intra_samples',
 'model_name',
 'cls_loss_fn',
#  'batch_size',
#  'num_epochs',
 'lr_init',
 'w_top_loss',
#  'weight_decay', 
 'weight_decay_feat_ext',
 'weight_decay_cls',
#  'w_top_loss_rampup_start',
#  'w_top_loss_rampup_end',
 'top_scale',
#  'num_branches', 
 'pers_type',
#  'compute_persistence',
 'tag'
}

stats_white_list = {
#     'lr', 
#     'batch_cls_loss', 
#     'batch_top_loss', 
    'acc_train', 
    'acc_test', 
}


def load_results():
    global RESULTS
    RESULTS = [LoggerReader(r) for r in glob.glob(str(root / '*'))]        
    RESULTS = sorted(RESULTS, key=lambda x: x.date)

def args_df_from_results():
    load_results()
    R = []
    
    for i, r in enumerate(RESULTS):
        
        df = {k: str(v) if isinstance(v, (list, tuple)) else v for  k, v in r.experiment_args.items() if k in args_white_list}
        df['date'] = r.date
        df = pd.DataFrame(df, index=[i])
        
        try: 
            df['progress'] = str(r.progress)
        except FileNotFoundError:
            pass              
        
        
        R.append(df)   
        
    return pd.concat(R, sort=False)

def df_from_results():
    A = args_df_from_results()
    
    tmp = []
    for i, r in enumerate(RESULTS):
        v = r.values_by_run
        df = {}
        for k in ['acc_train', 'acc_test']:
            df[k] = np.mean([x[k][-1] for x in v.values()])
            
        tmp.append(pd.DataFrame(df, index=[i]))
    
    B = pd.concat(tmp, sort=False)
    return A.join(B)  

In [7]:
df_from_results()

,num_intra_samples,model_name,cls_loss_fn,lr_init,w_top_loss,top_scale,weight_decay_cls,weight_decay_feat_ext,pers_type,tag,date,progress,acc_train,acc_test
0,16,"['SimpleCNN13', {'batch_norm': True, 'cls_spec...",CrossEntropyLoss,0.5,0.1,0.7,0.001,0.001,VrPersistenceL_2,,2020-09-29 12:23:59,"(10, 309)",100.0,71.512
